# Imports 

In [2]:
# Descargar dataset desde Kaggle
import kagglehub
import pandas as pd
import os
import shutil  
import warnings
# Importaciones para el manejo geoespacial
import geopandas as gpd
from shapely.geometry import Point
import subprocess 

# Desactivar advertencias de sjoin
warnings.filterwarnings('ignore', 'The CRS of the two GeoDataFrames are not the same. *')

# --- PROCESAMIENTO GEOESPACIAL ---

def assign_borough_from_geometry(df: pd.DataFrame, lat_col: str, lon_col: str, output_col: str) -> pd.DataFrame:
    """
    Asigna el distrito (borough) a las coordenadas usando una unión espacial.
    
    Argumentos:
        df (pd.DataFrame): DataFrame de entrada.
        lat_col (str): Nombre de la columna de latitud.
        lon_col (str): Nombre de la columna de longitud.
        output_col (str): Nombre de la nueva columna de distrito a crear.
    """
    print(f"\n--- Iniciando asignación de distrito para {output_col} ---")
    
    # 1. Cargar límites de distritos de NYC (Boroughs)
    nyc_boundaries_url = "https://data.cityofnewyork.us/resource/gthc-hcne.geojson"
    
    try:
        boroughs_gdf = gpd.read_file(nyc_boundaries_url)
        
        boroughs_gdf = boroughs_gdf.set_geometry('geometry')

        # Renombrar la columna del distrito en el GeoDataFrame de límites
        boroughs_gdf.rename(columns={'boroname': 'borough_name_geo'}, inplace=True)
        
        if boroughs_gdf.crs is None:
             boroughs_gdf.set_crs(epsg=4326, inplace=True)
        else:
             boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)

    except Exception as e:
        print(f"Error CRÍTICO al cargar o procesar los límites de los distritos: {e}")
        return df

    # 2. Convertir datos de Uber a GeoDataFrame
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    
    df_clean = df.dropna(subset=[lat_col, lon_col]).copy()
    
    if df_clean.empty:
        print("No hay coordenadas válidas para procesar.")
        df[output_col] = 'Out of NYC/Unknown'
        return df

    geometry = [Point(xy) for xy in zip(df_clean[lon_col], df_clean[lat_col])]
    points_gdf = gpd.GeoDataFrame(df_clean, geometry=geometry, crs="EPSG:4326")
    
    # 3. Realizar Unión Espacial (Spatial Join)
    if points_gdf.crs != boroughs_gdf.crs:
        boroughs_gdf = boroughs_gdf.to_crs(points_gdf.crs)

    join_result = gpd.sjoin(points_gdf, boroughs_gdf[['borough_name_geo', 'geometry']], how="left", predicate="within")
    
    # 4. Mapear el resultado de nuevo al DataFrame original
    # Usar el nombre de columna dinámico (output_col)
    df[output_col] = join_result['borough_name_geo'].reindex(df.index)
    
    # 5. Rellenar valores faltantes (coordenadas fuera de NYC o inválidas)
    df[output_col] = df[output_col].fillna('Out of NYC/Unknown')
    
    print(f"--- Asignación de distrito ({output_col}) completada ---")
    print(f"Top 5 de distritos encontrados para {output_col}:")
    print(df[output_col].value_counts().head(5))
    
    return df

# --- SCRIPT PRINCIPAL ---

# Descargar la versión más reciente
path = kagglehub.dataset_download("praveenluppunda/uber-dataset")

print("Path to dataset files:", path)

# Buscar archivo CSV en el directorio descargado
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("No se encontró ningún archivo CSV en el dataset.")
    
file_path = os.path.join(path, csv_files[0])
print("Archivo CSV encontrado:", file_path)

# Cargar dataset
df = pd.read_csv(file_path)

# --- TRANSFORMACIONES ---

df = df.head(10000).copy() 

# 0.1. FILTRO DE COORDENADAS (lat/lon ~ 0,0) (¡NUEVO!)
# Primero, asegurar que las coordenadas son numéricas para el filtro
df['pickup_latitude'] = pd.to_numeric(df['pickup_latitude'], errors='coerce')
df['pickup_longitude'] = pd.to_numeric(df['pickup_longitude'], errors='coerce')
df['dropoff_latitude'] = pd.to_numeric(df['dropoff_latitude'], errors='coerce')
df['dropoff_longitude'] = pd.to_numeric(df['dropoff_longitude'], errors='coerce')

# Definir el umbral (e.g., +/- 0.1 grados de latitud y longitud)
ZERO_THRESHOLD = 0.1 

# Crear máscaras para identificar registros que tienen un pickup o dropoff cerca de (0,0)
is_pickup_zero = (abs(df['pickup_latitude']) < ZERO_THRESHOLD) & (abs(df['pickup_longitude']) < ZERO_THRESHOLD)
is_dropoff_zero = (abs(df['dropoff_latitude']) < ZERO_THRESHOLD) & (abs(df['dropoff_longitude']) < ZERO_THRESHOLD)

# Combinar las máscaras: filtrar si CUALQUIERA de las ubicaciones está cerca de (0,0)
df_clean = df[~(is_pickup_zero | is_dropoff_zero)].copy()

# Mostrar cuántos registros se eliminaron
rows_removed = len(df) - len(df_clean)
print(f"\n--- Limpieza de datos (Coordenadas ~0,0) ---")
print(f"Registros iniciales: {len(df)}")
print(f"Registros eliminados (cerca de 0,0): {rows_removed}")
print(f"Registros restantes: {len(df_clean)}")

df = df_clean # Usar el DataFrame limpio para el resto de las transformaciones

# 0.2. Asignar distrito (borough) a las coordenadas
# Asignar distrito de SALIDA
if 'pickup_latitude' in df.columns and 'pickup_longitude' in df.columns:
    df = assign_borough_from_geometry(df, 'pickup_latitude', 'pickup_longitude', 'pickup_borough')
    
# Asignar distrito de LLEGADA
if 'dropoff_latitude' in df.columns and 'dropoff_longitude' in df.columns:
    df = assign_borough_from_geometry(df, 'dropoff_latitude', 'dropoff_longitude', 'dropoff_borough')
    
# 1. Eliminar columna VendorID
if 'VendorID' in df.columns:
    df = df.drop(columns=['VendorID'])

# 2. Calcular minutos de viaje
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['trip_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# 3. Convertir distancia de millas a kilómetros
if 'trip_distance' in df.columns:
    df['trip_distance_km'] = df['trip_distance'] * 1.60934
    df = df.drop(columns=['trip_distance'])

# 4. Transformar RatecodeID a categorías
ratecode_map = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride",
    99: "Null/unknown"
}
if 'RatecodeID' in df.columns:
    df['RatecodeID'] = df['RatecodeID'].map(ratecode_map)

# 5. Eliminar columna store_and_fwd_flag
if 'store_and_fwd_flag' in df.columns:
    df = df.drop(columns=['store_and_fwd_flag'])

# 6. Transformar payment_type a categorías
payment_type_map = {
    0: "Flex Fare trip",
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}
if 'payment_type' in df.columns:
    df['payment_type'] = df['payment_type'].map(payment_type_map)

# 7. Guardar el archivo final
output_file = "uber_dataset_con_distritos.csv"
df.to_csv(output_file, index=False)
print(f"\nArchivo procesado guardado como: {output_file}")

# 8. Eliminar el directorio temporal de descarga
shutil.rmtree(path)
print(f"Directorio temporal eliminado: {path}")

100%|██████████| 3.90M/3.90M [00:00<00:00, 4.88MB/s]

Extracting files...


Path to dataset files: C:\Users\aesto\.cache\kagglehub\datasets\praveenluppunda\uber-dataset\versions\1
Archivo CSV encontrado: C:\Users\aesto\.cache\kagglehub\datasets\praveenluppunda\uber-dataset\versions\1\uber_data.csv

--- Limpieza de datos (Coordenadas ~0,0) ---
Registros iniciales: 10000
Registros eliminados (cerca de 0,0): 74
Registros restantes: 9926

--- Iniciando asignación de distrito para pickup_borough ---
--- Asignación de distrito (pickup_borough) completada ---
Top 5 de distritos encontrados para pickup_borough:
pickup_borough
Manhattan             9259
Queens                 474
Brooklyn               181
Out of NYC/Unknown       6
Bronx                    6
Name: count, dtype: int64

--- Iniciando asignación de distrito para dropoff_borough ---
--- Asignación de distrito (dropoff_borough) completada ---
Top 5 de distritos encontrados para dropoff_borough:
dropoff_borough
Manhattan             9209
Queens                 461
Brooklyn               169
Bronx           

In [4]:
print(df.payment_type.value_counts())

payment_type
Credit card    7258
Cash           2658
No charge         7
Dispute           3
Name: count, dtype: int64
